In [1]:
!pip3 install sklearn

In [2]:
# import required libraries
import sys
sys.path.insert(0, '../../../BERT-FAQ/')

from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import ndcg_score
from shared.utils import load_from_json
from metric import NDCG
from tqdm import tqdm
import pandas as pd
import numpy as np
import os.path
import math

In [3]:
# define functions to compute NDCG@, P@, MAP 

def compute_map(result_filepath):
    query_results = load_from_json(result_filepath)

    sum_ap = 0
    num_queries = 0
    for result in query_results:
        query_string = result['query_string']
        topk_results = result['rerank_preds']

        labels = []
        reranks = []

        for topk in topk_results:
            labels.append(topk['label'])
            reranks.append(topk['score'])

        true_relevance = np.array(labels)
        scores = np.array(reranks)

        ap = 0
        all_zeros = not np.any(labels)
        if labels and reranks and not all_zeros:
            ap = average_precision_score(true_relevance, scores)

        sum_ap = sum_ap + ap
        num_queries = num_queries + 1

    return float(sum_ap / num_queries)

def compute_prec(result_filepath, k):
    query_results = load_from_json(result_filepath)

    sum_prec = 0
    num_queries = 0
    for result in query_results:
        query_string = result['query_string']
        topk_results = result['rerank_preds']

        labels = []
        reranks = []

        for topk in topk_results[:k]:
            labels.append(topk['label'])
            reranks.append(topk['score'])

        true_relevance = np.array(labels)
        scores = np.array(reranks)

        prec = 0
        all_zeros = not np.any(labels)
        if labels and reranks and not all_zeros:
            prec = sum(true_relevance) / len(true_relevance)

        sum_prec = sum_prec + prec
        num_queries = num_queries + 1

    return float(sum_prec / num_queries)

def compute_ndcg(result_filepath, k):
    query_results = load_from_json(result_filepath)

    sum_ndcg = 0
    num_queries = 0
    for result in query_results:
        query_string = result['query_string']
        topk_results = result['rerank_preds']

        labels = []
        reranks = []

        for topk in topk_results[:k]:
            labels.append(topk['label'])
            reranks.append(topk['score'])

        true_relevance = np.asarray([labels])
        scores = np.asarray([reranks])

        ndcg = 0
        if labels and reranks:
            ndcg = ndcg_score(true_relevance, scores)

        sum_ndcg = sum_ndcg + ndcg
        num_queries = num_queries + 1

    return float(sum_ndcg / num_queries)

In [4]:
# define function to generate evaluation DataFrame

def get_evaluation_df(rank_results_filepath):
    output = dict()
    metric_name = []

    i = 0

    rankers = ["unsupervised", "supervised"] 
    loss_type = ["triplet", "softmax"]
    query_types = ["faq", "user_query"]
    neg_types = ["simple", "hard"]

    topk = [3, 5, 10]

    for ranker in rankers:

        # Compute metrics for the unsupervised method
        if ranker == "unsupervised":
            file_path = rank_results_filepath + "/" + ranker

            method = ranker

            answer_metric = []
            question_metric = []
            question_answer_metric = []
            question_answer_concat_metric = []

            i += 1
            for k in topk:
                # compute NDCG@k
                if i == 1:
                    metric_name.append("ndcg@" + str(k))

                answer_metric.append(compute_ndcg(file_path + "/es_query_by_answer.json", k))
                question_metric.append(compute_ndcg(file_path + "/es_query_by_question.json", k))
                question_answer_metric.append(compute_ndcg(file_path + "/es_query_by_question_answer.json", k))
                question_answer_concat_metric.append(compute_ndcg(file_path + "/es_query_by_question_answer_concat.json", k))

                # compute P@k
                if i == 1:
                    metric_name.append("prec@" + str(k))

                answer_metric.append(compute_prec(file_path + "/es_query_by_answer.json", k))
                question_metric.append(compute_prec(file_path + "/es_query_by_question.json", k))
                question_answer_metric.append(compute_prec(file_path + "/es_query_by_question_answer.json", k))
                question_answer_concat_metric.append(compute_prec(file_path + "/es_query_by_question_answer_concat.json", k))

            # compute MAP
            if i == 1:
                metric_name.append("MAP" + str(k))

            answer_metric.append(compute_map(file_path + "/es_query_by_answer.json"))
            question_metric.append(compute_map(file_path + "/es_query_by_question.json"))
            question_answer_metric.append(compute_map(file_path + "/es_query_by_question_answer.json"))
            question_answer_concat_metric.append(compute_map(file_path + "/es_query_by_question_answer_concat.json"))
            

            output[method + "_answer"] = answer_metric
            output[method + "_question"] = question_metric
            output[method + "_question_answer"] = question_answer_metric
            output[method + "_question_answer_concat"] = question_answer_concat_metric

        else:
        # Compute metrics for the supervised method
            for loss in loss_type:
                for query_type in query_types:
                    
                    for neg_type in neg_types:
                        
                        file_path = rank_results_filepath + "/" + ranker + "/" + loss + "/" + query_type + "/" + neg_type
                        
                        if os.path.isdir(file_path):

                            method = loss + "_" + query_type + "_" + neg_type

                            answer_metric = []
                            question_metric = []
                            question_answer_metric = []
                            question_answer_concat_metric = []

                            for k in topk:

                                # compute NDCG@k
                                answer_metric.append(compute_ndcg(file_path + "/reranked_query_by_answer.json", k))
                                question_metric.append(compute_ndcg(file_path + "/reranked_query_by_question.json", k))
                                question_answer_metric.append(compute_ndcg(file_path + "/reranked_query_by_question_answer.json", k))
                                question_answer_concat_metric.append(compute_ndcg(file_path + "/reranked_query_by_question_answer_concat.json", k))

                                # compute P@k
                                answer_metric.append(compute_prec(file_path + "/reranked_query_by_answer.json", k))
                                question_metric.append(compute_prec(file_path + "/reranked_query_by_question.json", k))
                                question_answer_metric.append(compute_prec(file_path + "/reranked_query_by_question_answer.json", k))
                                question_answer_concat_metric.append(compute_prec(file_path + "/reranked_query_by_question_answer_concat.json", k))

                            # compute MAP
                            answer_metric.append(compute_map(file_path + "/reranked_query_by_answer.json"))
                            question_metric.append(compute_map(file_path + "/reranked_query_by_question.json"))
                            question_answer_metric.append(compute_map(file_path + "/reranked_query_by_question_answer.json"))
                            question_answer_concat_metric.append(compute_map(file_path + "/reranked_query_by_question_answer_concat.json"))
                            

                            output[method + "_answer"] = answer_metric
                            output[method + "_question"] = question_metric
                            output[method + "_question_answer"] = question_answer_metric
                            output[method + "_question_answer_concat"] = question_answer_concat_metric
                        else:
                            pass
                        
    data = dict()
    methods = []
    for method in output:
        metrics = output[method]
        metrics = ["{0:.4f}".format(x) for x in metrics]
        data[method] = metrics
        methods.append(method)

    # Generate evaluation DataFrame 
    df = pd.DataFrame.from_dict(data, orient='index')
    df.columns = ["NDCG@3", "NDCG@5","NDCG@10", "P@3",	"P@5", "P@10", "MAP"]
    df['Method'] = methods
    df = df[["Method", "NDCG@3", "NDCG@5","NDCG@10", "P@3",	"P@5", "P@10", "MAP"]]
    df.reset_index(drop=True, inplace=True)
    return df

In [5]:
# generate evaluation dataframe
rank_results_filepath="../../../BERT-FAQ/data/CovidFAQ/rank_results"
df = get_evaluation_df(rank_results_filepath)

In [6]:
df

,Method,NDCG@3,NDCG@5,NDCG@10,P@3,P@5,P@10,MAP
0,unsupervised_answer,0.2536,0.1221,0.2940,0.0985,0.3466,0.0730,0.2846
1,unsupervised_question,0.7844,0.3451,0.8104,0.2286,0.8147,0.1324,0.7422
2,unsupervised_question_answer,0.6976,0.3080,0.7246,0.2046,0.7465,0.1154,0.6604
3,unsupervised_question_answer_concat,0.5756,0.2588,0.6067,0.1846,0.6392,0.1073,0.5705
4,triplet_faq_simple_answer,0.4896,0.2427,0.5308,0.1740,0.5680,0.1030,0.4937
5,triplet_faq_simple_question,0.8142,0.3646,0.8357,0.2364,0.8383,0.1347,0.7840
6,triplet_faq_simple_question_answer,0.7455,0.3377,0.7674,0.2180,0.7903,0.1199,0.7209
7,triplet_faq_simple_question_answer_concat,0.6754,0.3089,0.7129,0.2119,0.7444,0.1194,0.6763
8,triplet_faq_hard_answer,0.5277,0.2616,0.5642,0.1872,0.5960,0.1108,0.5127
9,triplet_faq_hard_question,0.8249,0.3677,0.8450,0.2494,0.8497,0.1362,0.7946


In [7]:
# dump StackFAQ results to csv file
df.to_csv(rank_results_filepath + "/results.csv", index=False)